In [3]:
import numpy as np

#import tensorflow as tf
#from keras import layers
#from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Lambda 
#from keras.layers import Flatten, Conv2D, MaxPooling2D, AveragePooling2D, Reshape, LSTM, Embedding, TimeDistributed
#from keras.models import Model, load_model
#from keras.preprocessing import image
#from keras.initializers import glorot_uniform
from keras.optimizers import Adam
#import keras.backend as K

import preprocessing
import model

from matplotlib.pyplot import imshow
%matplotlib inline


First version of a Deep Captionning model using a CNN-LSTM architecture (Without attention. For now...)

See Show and Tell reference architecture here: 
    https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Vinyals_Show_and_Tell_2015_CVPR_paper.pdf
    
# 1 - Encoder: CNN Features extraction using a standard ResNet model

In [7]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = model.identity_block(A_prev, f = 2, filters = [2, 4, 6], stage = 1, block = 'a')
    test.run(tf.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    print("out = " + str(out[0][1][1][0]))

out = [ 0.1971682 -0.         1.3561226  2.1713073 -0.         1.3324987]


## Convolutional block
The ResNet "convolutional block" is used when the input and output dimensions don't match up. 
The difference with the identity block is that there is a CONV2D layer in the shortcut path:

<img src="img/convblock.png">

This image was borrowed from Coursera's <b> Convolutional Neural Networks </b> class by <b> Andrew Ng </b> 

The CONV2D layer in the shortcut path is used to resize the input  xx  to a different dimension, so that the dimensions match up in the final addition needed to add the shortcut value back to the main path. 

Here are the individual steps.

### First component of main path:

The first CONV2D has F1 filters of shape (1,1) and a stride of (s,s). Its padding is "valid" (none) and its name is conv_name_base + '2a'.
The first BatchNorm is normalizing the 'channels' axis. Its name is bn_name_base + '2a'.
The ReLU activation function has no name and no hyperparameters.

### Second component of main path:

The second CONV2D has F2 filters of shape (f,f) and a stride of (1,1). Its padding is "same" and it's name is conv_name_base + '2b'.
The second BatchNorm is normalizing the 'channels' axis. Its name is bn_name_base + '2b'.
The ReLU activation function has no name and no hyperparameters.

### Third component of main path:

The third CONV2D has F3 filters of shape (1,1) and a stride of (1,1). Its padding is "valid" and it's name is conv_name_base + '2c'.
The third BatchNorm is normalizing the 'channels' axis. Its name is bn_name_base + '2c'.

### Shortcut path:

The CONV2D has F3 filters of shape (1,1) and a stride of (s,s). Its padding is "valid" and its name is conv_name_base + '1'.
The BatchNorm is normalizing the 'channels' axis. Its name is bn_name_base + '1'.

### Final step:

The shortcut and the main path values are added together then a ReLU activation function is applied.

In [8]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = model.convolutional_block(A_prev, f = 2, filters = [2, 4, 6], stage = 1, block = 'a')
    test.run(tf.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    print("out = " + str(out[0][1][1][0]))

out = [ 0.09018463  1.2348979   0.46822023  0.03671761 -0.          0.65516603]


### ResNet model (50 layers)

<img src="img/resnet.png">

This image was borrowed from Coursera's Convolutional Neural Networks class by Andrew Ng

Architecture of this ResNet-50 model:

Zero-padding pads the input with a pad of (3,3)
## Stage 1:
The 2D Convolution has 64 filters of shape (7,7) and uses a stride of (2,2). Its name is "conv1".
BatchNorm is applied to the 'channels' axis of the input.
MaxPooling uses a (3,3) window and a (2,2) stride.
## Stage 2:
The convolutional block uses three sets of filters of size [64,64,256], "f" is 3, "s" is 1 and the block is "a".
The 2 identity blocks use three sets of filters of size [64,64,256], "f" is 3 and the blocks are "b" and "c".
## Stage 3:
The convolutional block uses three sets of filters of size [128,128,512], "f" is 3, "s" is 2 and the block is "a".
The 3 identity blocks use three sets of filters of size [128,128,512], "f" is 3 and the blocks are "b", "c" and "d".
## Stage 4:
The convolutional block uses three sets of filters of size [256, 256, 1024], "f" is 3, "s" is 2 and the block is "a".
The 5 identity blocks use three sets of filters of size [256, 256, 1024], "f" is 3 and the blocks are "b", "c", "d", "e" and "f".
## Stage 5:
The convolutional block uses three sets of filters of size [512, 512, 2048], "f" is 3, "s" is 2 and the block is "a".
The 2 identity blocks use three sets of filters of size [512, 512, 2048], "f" is 3 and the blocks are "b" and "c".
The 2D Average Pooling uses a window of shape (2,2) and its name is "avg_pool".
The 'flatten' layer doesn't have any hyperparameters or name.
The Fully Connected (Dense) layer reduces its input to the number of classes using a softmax activation. Its name is 'fc' + str(classes).

# 2 - Decoder: LSTM based caption generator

Insert diagram and documentation here

# 3 - Model

In [4]:
X, Y  = preprocessing.batch_generator("train", 0, 100)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [5]:
X["image_input"].shape

(90, 300, 300, 3)

In [2]:
mymodel = model.TrainShowAndTell(35, 60, 768)
mymodel.summary()
mymodel.compile(loss='categorical_crossentropy', optimizer=Adam(lr = 0.01), metrics=['accuracy'])














(?, 60)
(?, 1, 768)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
(?, 35, 768)
(?, 35, 768)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_input (InputLayer)        (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 306, 306, 3)  0           image_input[0][0]                
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 150, 150, 64) 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (N

NameError: name 'Adam' is not defined

In [6]:
def get_model_memory_usage(batch_size, model):
    import numpy as np
    from keras import backend as K

    shapes_mem_count = 0
    internal_model_mem_count = 0
    for l in model.layers:
        layer_type = l.__class__.__name__
        if layer_type == 'Model':
            internal_model_mem_count += get_model_memory_usage(batch_size, l)
        single_layer_mem = 1
        try:
            for s in l.output_shape:
                if s is None:
                    continue
                single_layer_mem *= s
            shapes_mem_count += single_layer_mem
        except AttributeError:
            continue

    trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
    non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

    number_size = 4.0
    if K.floatx() == 'float16':
         number_size = 2.0
    if K.floatx() == 'float64':
         number_size = 8.0

    total_memory = number_size*(batch_size*shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3) + internal_model_mem_count
    return gbytes

In [7]:
get_model_memory_usage(100, mymodel)

16.612

In [8]:
import gc

In [11]:
gc.collect()

20

In [20]:
a0 = np.zeros((X["image_input"].shape[0], 60))
c0 = np.zeros((X["image_input"].shape[0], 60))

In [1]:
mymodel.fit([X["image_input"],X["caption_input"]
             , a0, c0], Y["output"], batch_size=X["image_input"].shape[0], epochs=5, verbose=2)

NameError: name 'mymodel' is not defined

In [21]:
a0.shape

(90, 60)